---

# Atenção!

Lembre-se de clonar este notebook antes de tentar editar as células de código.

Para isso, basta seguir os passos:

File -> Save a copy in Drive



---

In [ ]:
!pip install ortools==9.0.9048

# Exercício 1



In [ ]:
from ortools.constraint_solver import pywrapcp


def solve_mtsp_ortools(distance_matrix, num_vehicles=1):
    n = distance_matrix.shape[0]  # número de nós do problema
    depot_node = 0  # número do nó que representa o ponto de origem
    manager = pywrapcp.RoutingIndexManager(n, num_vehicles, depot_node)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(i, j):
        # `i` e `j` são índices internos do OR-Tools. Precisamos primeiro
        # convertê-los em nós do nosso problema
        ni = manager.IndexToNode(i)
        nj = manager.IndexToNode(j)
        return distance_matrix[ni, nj]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Resolve o problema com métodos default
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    solution = routing.SolveWithParameters(search_parameters)

    # Constrói as rotas finais
    def create_vehicle_route(vehicle_index):
        route = []
        index = routing.Start(vehicle_index)
        node = manager.IndexToNode(index)
        route.append(node)

        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            node = manager.IndexToNode(index)
            route.append(node)
        return route

    routes = []
    for vehicle_index in range(num_vehicles):
        routes.append(create_vehicle_route(vehicle_index))

    return routes, solution.ObjectiveValue()

Após construir a matriz de distâncias, temos os seguintes resultados:

In [ ]:
import numpy as np


distance_matrix = np.array(
    [
        [0, 7, 7, 8, 5, 6],
        [7, 0, 5, 105, 115, 100],
        [7, 5, 0, 105, 110, 110],
        [8, 105, 105, 0, 100, 115],
        [5, 115, 110, 100, 0, 11],
        [6, 100, 110, 115, 11, 0],
    ]
)


routes1, distance1 = solve_mtsp_ortools(distance_matrix, num_vehicles=1)
routes2, distance2 = solve_mtsp_ortools(distance_matrix, num_vehicles=2)
routes3, distance3 = solve_mtsp_ortools(distance_matrix, num_vehicles=3)
routes4, distance4 = solve_mtsp_ortools(distance_matrix, num_vehicles=4)

print(f"mTSP com 1 veículo. Rotas: {routes1}, Distância total: {distance1}")
print(f"mTSP com 2 veículos. Rotas: {routes2}, Distância total: {distance2}")
print(f"mTSP com 3 veículos. Rotas: {routes3}, Distância total: {distance3}")
print(f"mTSP com 4 veículos. Rotas: {routes4}, Distância total: {distance4}")

Observe como a distância total das rotas é reduzida com o acréscimo de caixeiros. Veja também que de 3 em diante não há mais diferença, o que significaria que a distância total poderia até aumentar neste caso.

# Exercício 2

Aqui está um programa que usa a função anterior:

In [ ]:
def find_best_number_of_salespeople(distance_matrix):
    # Inicializa a melhor rota com um número muito alto
    best_routes = []
    best_distance = np.inf
    best_num_vehicles = 0

    n = distance_matrix.shape[0]  # número de nós

    # Observe que não faz sentido haver mais caixeiros que nós, então podemos
    # iterar para no máximo `n` caixeiros
    for num_vehicles in range(1, n):
        routes, distance = solve_mtsp_ortools(distance_matrix, num_vehicles)
        # Se a distância retornada não for menor que a que temos como melhor,
        # interrompa o processo
        if distance >= best_distance:
            break

        # Atualiza os resultados atuais como melhores
        best_routes = routes
        best_distance = distance
        best_num_vehicles = num_vehicles

    return best_routes, best_distance, best_num_vehicles

Vamos testá-lo com o problema anterior:

In [ ]:
distance_matrix = np.array(
    [
        [0, 7, 7, 8, 5, 6],
        [7, 0, 5, 105, 115, 100],
        [7, 5, 0, 105, 110, 110],
        [8, 105, 105, 0, 100, 115],
        [5, 115, 110, 100, 0, 11],
        [6, 100, 110, 115, 11, 0],
    ]
)

find_best_number_of_salespeople(distance_matrix)

Observe como obtemos o mesmo resultado do exercício anterior.

Agora vamos testar com uma matriz aleatória:

In [ ]:
n = 100

# Inicializa a semente como 1 para que sempre geremos a mesma matriz
np.random.seed(1)
distance_matrix = np.random.randint(0, 200, size=(n, n))

find_best_number_of_salespeople(distance_matrix)

Aqui valeu mais pena usar apenas um caixeiro, já que a primeira rota foi `[0, 0]`.

# Exercício 3

In [ ]:
from ortools.constraint_solver import pywrapcp


def solve_vrp_ortools(
    distance_matrix, node_demands, vehicle_capacity, num_vehicles=1
):
    n = distance_matrix.shape[0]  # número de nós do problema
    depot_node = 0  # número do nó que representa o ponto de origem
    manager = pywrapcp.RoutingIndexManager(n, num_vehicles, depot_node)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(i, j):
        # `i` e `j` são índices internos do OR-Tools. Precisamos primeiro
        # convertê-los em nós do nosso problema
        ni = manager.IndexToNode(i)
        nj = manager.IndexToNode(j)
        return distance_matrix[ni, nj]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Adiciona a restrição de capacidade
    def demand_callback(from_index):
        """Retorna a demanda de um nó"""
        from_node = manager.IndexToNode(from_index)
        return node_demands[from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback
    )
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        [vehicle_capacity] * num_vehicles,
        True,  # start cumul to zero
        'Capacity'
    )

    # Resolve o problema com métodos default
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    solution = routing.SolveWithParameters(search_parameters)

    # Constrói as rotas finais
    def create_vehicle_route(vehicle_index):
        route = []
        index = routing.Start(vehicle_index)
        node = manager.IndexToNode(index)
        route.append(node)

        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            node = manager.IndexToNode(index)
            route.append(node)
        return route

    routes = []
    for vehicle_index in range(num_vehicles):
        routes.append(create_vehicle_route(vehicle_index))

    return routes, solution.ObjectiveValue()

O primeiro passo é determinar os dados do problema, como a matriz de distâncias e as demandas de cada nó.
Em seguida, para cada capacidade, experimente resolver o problema para números diferentes de veículos até que obtenhamos uma solução sem erro.

In [ ]:
distance_matrix = np.array(
    [
        [0, 7, 7, 8, 5, 6],
        [7, 0, 5, 10, 22, 5],
        [7, 5, 0, 6, 16, 15],
        [8, 10, 6, 0, 6, 20],
        [5, 22, 16, 6, 0, 11],
        [6, 5, 15, 20, 11, 0],
    ]
)
node_demands = [0, 4, 5, 5, 7, 2]

Por exemplo, com capacidade 7, temos um erro quando usamos menos que 4 veículos:

In [ ]:
# Experimente testar com outros números
# Capacidade 7
vehicle_capacity = 7
routes, distance = solve_vrp_ortools(distance_matrix, node_demands, vehicle_capacity, num_vehicles=1)
print(f"Capacidade {vehicle_capacity}; Distância total: {distance}")

Repetindo a análise para cada capacidade, temos os seguintes resultados:

In [ ]:
# Capacidade 7
vehicle_capacity = 7
routes, distance = solve_vrp_ortools(distance_matrix, node_demands, vehicle_capacity, num_vehicles=4)
print(f"Capacidade {vehicle_capacity}; Distância total: {distance}")

# Capacidade 10
vehicle_capacity = 10
routes, distance = solve_vrp_ortools(distance_matrix, node_demands, vehicle_capacity, num_vehicles=3)
print(f"Capacidade {vehicle_capacity}; Distância total: {distance}")

# Capacidade 20
vehicle_capacity = 20
routes, distance = solve_vrp_ortools(distance_matrix, node_demands, vehicle_capacity, num_vehicles=2)
print(f"Capacidade {vehicle_capacity}; Distância total: {distance}")

# Capacidade 100
vehicle_capacity = 100
routes, distance = solve_vrp_ortools(distance_matrix, node_demands, vehicle_capacity, num_vehicles=1)
print(f"Capacidade {vehicle_capacity}; Distância total: {distance}")

Observe como com o aumento da capacidade obtemos soluções cada vez melhores. No limite quando a capacidade de um veículo é grande o suficiente, nós acabamos resolvendo um TSP. Experimente voltar na aula anterior e comparar a solução com o caso de capacidade de 100.

# Exercício 4

Aqui está uma função que resolve estes problemas. Observe como não precisamos mais passar `num_vehicles` como variável de entrada.

In [ ]:
from ortools.constraint_solver import pywrapcp


def solve_vrp_ortools2(
    distance_matrix, node_demands, vehicle_capacity
):
    n = distance_matrix.shape[0]  # número de nós do problema
    depot_node = 0  # número do nó que representa o ponto de origem

    # Vamos usar `n` como número de veículos, pois haveria na pior das hipóteses
    # um veículo entregando cada pacote
    num_vehicles = n
    manager = pywrapcp.RoutingIndexManager(n, num_vehicles, depot_node)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(i, j):
        # `i` e `j` são índices internos do OR-Tools. Precisamos primeiro
        # convertê-los em nós do nosso problema
        ni = manager.IndexToNode(i)
        nj = manager.IndexToNode(j)
        return distance_matrix[ni, nj]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Adiciona a restrição de capacidade
    def demand_callback(from_index):
        """Retorna a demanda de um nó"""
        from_node = manager.IndexToNode(from_index)
        return node_demands[from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback
    )
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        [vehicle_capacity] * num_vehicles,
        True,  # start cumul to zero
        'Capacity'
    )

    # Resolve o problema com métodos default
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    solution = routing.SolveWithParameters(search_parameters)

    # Caso não haja solução factível, retorne uma lista vazia como rotas e o
    # valor -1 como distância total
    if not solution:
        return [], -1

    # Constrói as rotas finais
    def create_vehicle_route(vehicle_index):
        route = []
        index = routing.Start(vehicle_index)
        node = manager.IndexToNode(index)
        route.append(node)

        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            node = manager.IndexToNode(index)
            route.append(node)
        return route

    routes = []
    for vehicle_index in range(num_vehicles):
        # Adicione apenas as rotas com mais que apenas [0, 0], ou seja, apenas
        # aquelas com ao menos três pontos
        route = create_vehicle_route(vehicle_index)
        if len(route) > 2:
            routes.append(route)

    return routes, solution.ObjectiveValue()

Vamos testar resolvendo o mesmo problema de antes:

In [ ]:
import numpy as np


distance_matrix = np.array(
    [
        [0, 7, 7, 8, 5, 6],
        [7, 0, 5, 10, 22, 5],
        [7, 5, 0, 6, 16, 15],
        [8, 10, 6, 0, 6, 20],
        [5, 22, 16, 6, 0, 11],
        [6, 5, 15, 20, 11, 0],
    ]
)
node_demands = [0, 4, 5, 5, 7, 2]

# Capacidade 7
vehicle_capacity = 7
routes, distance = solve_vrp_ortools2(distance_matrix, node_demands, vehicle_capacity)
print(f"Capacidade {vehicle_capacity}; Distância total: {distance}, Número de veículos: {len(routes)}")

# Capacidade 10
vehicle_capacity = 10
routes, distance = solve_vrp_ortools2(distance_matrix, node_demands, vehicle_capacity)
print(f"Capacidade {vehicle_capacity}; Distância total: {distance}, Número de veículos: {len(routes)}")

# Capacidade 20
vehicle_capacity = 20
routes, distance = solve_vrp_ortools2(distance_matrix, node_demands, vehicle_capacity)
print(f"Capacidade {vehicle_capacity}; Distância total: {distance}, Número de veículos: {len(routes)}")

# Capacidade 100
vehicle_capacity = 100
routes, distance = solve_vrp_ortools2(distance_matrix, node_demands, vehicle_capacity)
print(f"Capacidade {vehicle_capacity}; Distância total: {distance}, Número de veículos: {len(routes)}")

Observe como obtivemos os mesmos resultados de antes sem o problema de tentativa e erro.

Finalmente, para experimentar um caso em que não há solução factível, vamos colocar a capacidade do veículo menor que a maior demanda:

In [ ]:
# Capacidade 5 (a maior entrega não caberia em nenhum veículo)
vehicle_capacity = 5
routes, distance = solve_vrp_ortools2(distance_matrix, node_demands, vehicle_capacity)
print(f"Capacidade {vehicle_capacity}; Distância total: {distance}, Número de veículos: {len(routes)}")

Observe como obtemos uma resposta mais amigável em vez do erro anterior.

# Exercício 5

Aqui está uma função com o formato sugerido no enunciado. Ela consiste em duas subfunções para cada critério avaliado. Analise bem o código.

In [ ]:
def evaluate_solution(routes, node_demands, vehicle_capacity):
    """Avalia se a solução de um solver VRP é factível"""

    # O número de nós do problema é o tamanho da lista de demandas
    n = len(node_demands)
    is_feasible1 = _has_all_nodes(routes, n)
    is_feasible2 = _capacities_are_respected(
        routes, node_demands, vehicle_capacity
    )

    # Para a solução ser factível, as duas condições devem ser verdadeiras
    return is_feasible1 and is_feasible2


def _has_all_nodes(routes, n):
    """Verifica se um conjunto de rotas possui todos os elementos
    Se um problema tem, por exemplo, 5 nós, eles são numerados de 0 a 4. Logo,
    no caso geral, devemos coletar todos os nós de cada rota, agrupá-los sem
    repetição, e verificar se todos os nós de 0 a n - 1 estão presentes.
    """
    all_nodes = []
    for route in routes:
        all_nodes += route  # concatena cada rota separadamente

    # Para evitar repetições, vamos transformar a lista `all_nodes` em um
    # conjunto
    all_nodes_set = set(all_nodes)

    # Devemos verificar se este conjunto é igual ao conjunto {0, 1, ..., n - 1}
    expected_nodes_set = set(range(n))

    return all_nodes_set == expected_nodes_set


def _capacities_are_respected(routes, node_demands, vehicle_capacity):
    """Verifica se a demanda total de cada rota não viola a capacidade"""

    def compute_route_total_demand(route, node_demands):
        """Calcula a demanda total de uma rota"""
        total_demand = 0
        for node in route:
            total_demand += node_demands[node]
        return total_demand

    # Itera em cada rota. Se alguma violar a restrição de capacidade, retorne
    # `False`. Se passarmos por todas as rotas sem interrupção, significa que
    # todas respeitaram a restrição, e assim podemos retornar `True`
    for route in routes:
        if compute_route_total_demand(route, node_demands) > vehicle_capacity:
            return False

    return True

Vamos testar nosso código com os seguintes problemas

In [ ]:
import numpy as np


vehicle_capacity = 40

# n = 10
n = 10

# Inicializa a semente como 1 para que sempre geremos a mesma matriz
np.random.seed(1)
distance_matrix = np.random.randint(1, 101, size=(n, n))  # números de 1 a 100
node_demands = np.random.randint(1, 21, size=n)
node_demands[0] = 0  # primeiro nó é o ponto de origem

routes, distance = solve_vrp_ortools2(distance_matrix, node_demands, vehicle_capacity)
is_feasible = evaluate_solution(routes, node_demands, vehicle_capacity)

print(f"Problema com {n} nós, solução factível? {is_feasible}")

# n = 20
n = 20

# Inicializa a semente como 1 para que sempre geremos a mesma matriz
np.random.seed(1)
distance_matrix = np.random.randint(1, 101, size=(n, n))  # números de 1 a 100
node_demands = np.random.randint(1, 21, size=n)
node_demands[0] = 0  # primeiro nó é o ponto de origem

routes, distance = solve_vrp_ortools2(distance_matrix, node_demands, vehicle_capacity)
is_feasible = evaluate_solution(routes, node_demands, vehicle_capacity)

print(f"Problema com {n} nós, solução factível? {is_feasible}")

# n = 50
n = 50

# Inicializa a semente como 1 para que sempre geremos a mesma matriz
np.random.seed(1)
distance_matrix = np.random.randint(1, 101, size=(n, n))  # números de 1 a 100
node_demands = np.random.randint(1, 21, size=n)
node_demands[0] = 0  # primeiro nó é o ponto de origem

routes, distance = solve_vrp_ortools2(distance_matrix, node_demands, vehicle_capacity)
is_feasible = evaluate_solution(routes, node_demands, vehicle_capacity)

print(f"Problema com {n} nós, solução factível? {is_feasible}")

# n = 100
n = 100

# Inicializa a semente como 1 para que sempre geremos a mesma matriz
np.random.seed(1)
distance_matrix = np.random.randint(1, 101, size=(n, n))  # números de 1 a 100
node_demands = np.random.randint(1, 21, size=n)
node_demands[0] = 0  # primeiro nó é o ponto de origem

routes, distance = solve_vrp_ortools2(distance_matrix, node_demands, vehicle_capacity)
is_feasible = evaluate_solution(routes, node_demands, vehicle_capacity)

print(f"Problema com {n} nós, solução factível? {is_feasible}")

Logo, está tudo bem com nosso solver. Experimente aumentar ainda mais o número de nós e ver até que ponto ele deixaria de funcionar corretamente.